In [1]:
import tensorflow as tf
import numpy as np
import img_pross_help as imgh
import tf_helper_funcs as tf_help

In [2]:
# Dictionary for Image Encoding
key_dict = {
    'laptop':[0,0,1],
    'cup':[0,1,0],
    'chair':[0,1,1],
    'car':[1,0,0]
}
# Data Pre-process
img_wh = 256 # Image Width and Height
img_flat = img_wh**2 # The size of the image flattened
img_shape = (img_wh,img_wh) # Holds the dimension of the image
color_channels = 3 # RGB is a 3 color channel
num_classes = 3 # 4 Total number of classification possibilities
imgh.PrepImages("images/all_source_images","images/train_imgs", "images/test_imgs",img_wh,img_wh,0.6) #Only needs to be ran initially

In [3]:
train_Images,train_Labels = imgh.ImportImages("images/train_imgs",key_dict) # Extract Training Images and Labels
test_Images,test_Labels = imgh.ImportImages("images/test_imgs",key_dict)# Extract Testing Images and Labels

#train_Images = imgh.shape_up_X(train_Images,img_wh,color_channels,train_Labels.shape[0])
#test_Images = imgh.shape_up_X(test_Images,img_wh,color_channels,test_Labels.shape[0])

print("Training Label Shape: ",train_Labels.shape)
print("Training Images Shape:",train_Images.shape)
print(" ")
print("Testing Label Shape: ",test_Labels.shape)
print("Testing Images Shape:",test_Images.shape)
# Randomize Data
for i in range(10):
    r = np.random.permutation(range(len(train_Labels)))
    train_Labels = train_Labels[r,:]
    train_Images = train_Images[r,:]

Training Label Shape:  (323, 3)
Training Images Shape: (323, 256, 256, 3)
 
Testing Label Shape:  (312, 3)
Testing Images Shape: (312, 256, 256, 3)


In [4]:
print("Training Label Example: ")
train_Labels[0:10]

Training Label Example: 


array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1]])

### Model Overview:
An initial convolutional layer of size 28x28x10 and a filter size of 5x5 which will then go through a convolutional layer of size 14x14x15 with a filter size of 3 and using max pooling.

A fully connected layer will then flatten out the resulting convlutional outputs.


In [5]:
X = tf.placeholder(tf.float32, shape=[None,img_wh, img_wh, color_channels]) # X Input Site
x_image = tf.reshape(X, [-1, img_wh, img_wh, color_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_classes]) # Class Output Variable

In [6]:
# Create First Convolutional Layer
CONV_L1, W_L1 = tf_help.new_conv_layer(x_image, color_channels, 16, 60,False)

# Create Second Convolutional Layer
CONV_L2, W_L2 = tf_help.new_conv_layer(CONV_L1, 60, 8, 30)

# Create Third Convolutional Layer with Pooling
CONV_L3, W_L3 = tf_help.new_conv_layer(CONV_L2, 30, 5, 20)
# Flatten Second Convolutional Layer and Create Fully Connected Layer

flt_layer, features = tf_help.flatten_layer(CONV_L3)
print("Features: ", features)
FC_L1 = tf_help.fc_layer(flt_layer,features,512)

# Create Another Fully Connected Layer

FC_L2 = tf_help.fc_layer(FC_L1,512,num_classes,False)


Features:  81920


In [7]:
CONV_L1

<tf.Tensor 'Relu:0' shape=(?, 256, 256, 60) dtype=float32>

In [8]:
CONV_L2

<tf.Tensor 'Relu_1:0' shape=(?, 128, 128, 30) dtype=float32>

In [9]:
CONV_L3

<tf.Tensor 'Relu_2:0' shape=(?, 64, 64, 20) dtype=float32>

In [10]:
FC_L1

<tf.Tensor 'Relu_3:0' shape=(?, 512) dtype=float32>

In [11]:
FC_L2

<tf.Tensor 'add_4:0' shape=(?, 3) dtype=float32>

In [12]:
                                # Build and Connect Model
# Y
y_out = tf.nn.softmax(FC_L2) # Prototype Y output vector
#y_class = tf.argmax(y_out,axis=1) # Keep Largest Element of Vector
y_class = y_out
# Cost Function
cost_fun = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = FC_L2, labels = Y))

# Cost Optimization
optimize_fun = tf.train.GradientDescentOptimizer(0.02).minimize(cost_fun)
#optimize_fun = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost_fun)

# Accuracy Measurements
correct_ =  tf.equal(tf.cast(y_out, tf.float32), Y)
accuracy = tf.reduce_mean(tf.cast(correct_, tf.float32))

In [13]:
# Init Session
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
# Define Batch Size for Train Step
batch_size = 8

In [14]:
                                # Training Step
for i in range(100):
    r = np.random.permutation(range(len(train_Labels)))
    train_Labels = train_Labels[r,:]
    train_Images = train_Images[r,:]
    for j in range(len(train_Images),batch_size):
        img_set = train_Images[j,:]
        print(len(img_set))
        lab_set = train_Labels[j,:]
        opt_res = session.run(optimize_fun,feed_dict = {X:img_set, Y:lab_set})
        print(opt_res)

In [21]:
p = accuracy.eval(feed_dict = {X:test_Images[0:50], Y:test_Labels[0:50]})

test = session.run(y_out,feed_dict = {X:test_Images[0:10], Y:test_Labels[0:10]})
print(p)
print(np.round(test))
print(test_Labels[0:10])


1.0
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]
